In [130]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from random import randint
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep

## Obtaining the web page

In [83]:
url = 'https://www.billboard.com/charts/hot-100'

In [84]:
response = requests.get(url)
response.status_code

200

In [85]:
soup = BeautifulSoup(response.content, "html.parser")

## Creating the dataframe

In [86]:
titles = []
for title in soup.select("h3")[4:104]:
    titles.append(title.get_text().strip())

In [87]:
artists = []
for artist in soup.select("li.o-chart-results-list__item.\/\/.lrv-u-flex-grow-1.lrv-u-flex.lrv-u-flex-direction-column.lrv-u-justify-content-center.lrv-u-border-b-1.u-border-b-0\@mobile-max.lrv-u-border-color-grey-light.lrv-u-padding-l-1\@mobile-max > span"):
    artists.append(artist.get_text().strip())

In [88]:
#artists

In [89]:
top_100 = pd.DataFrame({"artist": artists, "song": titles})
top_100

,artist,song
0,Taylor Swift,All Too Well (Taylor's Version)
1,Adele,Easy On Me
2,The Kid LAROI & Justin Bieber,Stay
3,Lil Nas X & Jack Harlow,Industry Baby
4,Silk Sonic (Bruno Mars & Anderson .Paak),Smokin Out The Window
...,...,...
95,Swedish House Mafia & The Weeknd,Moth To A Flame
96,Cody Johnson,'Til You Can't
97,Doja Cat,Woman
98,The Weeknd,Take My Breath


## Obtaining the second list

In [90]:
url1 = 'https://www.elportaldemusica.es/lists/top-100-canciones/2021/45'

In [91]:
response1 = requests.get(url1)
response1.status_code

200

In [92]:
soup1 = BeautifulSoup(response1.content, "html.parser")

In [93]:
#soup1

In [94]:
artists1 = []
for name in soup1.select("div.related"):
    artists1.append(name.get_text().strip())

In [95]:
songs1 = []
for name in soup1.select("div.name"):
    songs1.append(name.get_text().strip())

In [96]:
second_top = pd.DataFrame({"artist": artists1, "song": songs1})
second_top

,artist,song
0,"C. Tangana, Nathy Peluso",Ateo
1,"zzoilo, Aitana",Mon Amour (remix)
2,BIZARRAP / ANUEL AA,ANUEL AA: BZRP MUSIC SESSION: VOL. 46
3,MYKE TOWERS,EXPERIMENTO
4,"Rvssian, Rauw Alejandro & Chris Brown",Nostálgico
...,...,...
95,KAROL G,SEJODIOTO
96,Olivia Rodrigo,Good 4 u
97,BIZARRAP / NICKY JAM,"NICKY JAM: BZRP MUSIC SESSIONS, VOL. 41"
98,"Jhay Cortez, Skrillex",En Mi Cuarto


### Asking a song name

In [97]:
def recommending_song(song):
    songs_to_recommend_top_100 = []
    position_top_100 = []
    songs_to_recommend_second_top = []
    position_second_top = []
    for i in range(100):
        if song == top_100['song'][i]:
            songs_to_recommend_top_100.append(top_100['song'][i])
            position_top_100.append(i)
        elif song == second_top['song'][i]:
            songs_to_recommend_second_top.append(second_top['song'][i])
            position_second_top.append(i)
    if len(songs_to_recommend_top_100)> 0:
        random_number = position_top_100[0]
        while random_number == position_top_100[0]:
            random_number = randint(0,99)
        i_recommend = top_100['song'][random_number]
        return print("Good choice! This song is now in the position number " + str(position_top_100[0]+1) + " of our first top from " + top_100['artist'][position_top_100[0]] + ". Here we have another recommendation of this top 100 songs: " + i_recommend + " from " + top_100['artist'][random_number])
    elif len(songs_to_recommend_second_top)> 0:
        random_number = position_second_top[0]
        while random_number == position_second_top[0]:
            random_number = randint(0,99)
        i_recommend = second_top['song'][random_number]
        return print("Good choice! This song is now in the position number " + str(position_second_top[0]+1) + " of our second top from " + second_top['artist'][position_second_top[0]] + ". Here we have another recommendation of this top 100 songs: " + i_recommend + " from " + second_top['artist'][random_number])
    else:
        print("Sorry, now we can't help you with that song, but if you want to try again with another one...")
        return 0

In [98]:
def recommending_artist(artist):
    songs_to_recommend_top_100 = []
    position_top_100 = []
    songs_to_recommend_second_top = []
    position_second_top = []
    for i in range(100):
        text1 = top_100['artist'][i]
        text2 = second_top['artist'][i]
        pattern = artist
        number_1 = re.findall(pattern, text1)
        number_2 = re.findall(pattern, text2)
        if len(number_1)> 0:
            songs_to_recommend_top_100.append(top_100['song'][i])
            position_top_100.append(i)
            number_1.pop(0)
        if len(number_2)> 0:
            songs_to_recommend_second_top.append(second_top['song'][i])
            position_second_top.append(i)
            number_2.pop(0)
    if len(songs_to_recommend_top_100)> 0:
        print('Good choice! This artist has ' + str(len(songs_to_recommend_top_100)) + ' songs:')
        for i in range(len(songs_to_recommend_top_100)):
            print('-' + songs_to_recommend_top_100[i] + ' in position ' + str(position_top_100[i] + 1))
        return 
    elif len(songs_to_recommend_second_top)> 0:
        print('Good choice! This artist has ' + str(len(songs_to_recommend_second_top)) + ' songs:')
        for i in range(len(songs_to_recommend_second_top)):
            print('-' + songs_to_recommend_second_top[i] + ' in position ' + str(position_second_top[i] + 1))
        return 
    else:
        print("Sorry, now we can't help you with that artist, but if you want to try again with another one...")
        return 0

In [99]:
def asking_song():
    print("Which song do you want?")
    song_name = input()
    value = recommending_song(song_name)
    while value == 0:
        print("Which new song do you want?")
        song_name = input()
        value = recommending_song(song_name)
    return

In [100]:
def asking_artist():
    print("Which artist do you want?")
    artist_name = input()
    value = recommending_artist(artist_name)
    while value == 0:
        print("Which new artist do you want?")
        artist_name = input()
        value = recommending_artist(song_name)
    return

In [101]:
def asking():
    print("If you want to search by song enter 1, and if you want to search by artist enter 2")
    search = input()
    while (search != '1') & (search != '2'):
        print(type(search))
        print("This isn't a valid value... Try again")
        print("If you want to search by song enter 1, and if you want to search by artist enter 2")
        search = input()
    if search == '1':
        return asking_song()
    elif search == '2':
        return asking_artist()

In [102]:
asking()

If you want to search by song enter 1, and if you want to search by artist enter 2


 sals


<class 'str'>
This isn't a valid value... Try again
If you want to search by song enter 1, and if you want to search by artist enter 2


 3


<class 'str'>
This isn't a valid value... Try again
If you want to search by song enter 1, and if you want to search by artist enter 2


 2


Which artist do you want?


 Tangana


Good choice! This artist has 2 songs:
-Ateo in position 1
-Tú Me Dejaste De Querer in position 53


### Opening Spotipy

In [103]:
secrets_file = open("secrets.txt","r")

In [104]:
string = secrets_file.read()

In [106]:
#string.split('\n')

In [107]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [108]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

### Obtaining different playlists

In [ ]:
37i9dQZEVXbJwoKy8qKpHG
37i9dQZEVXbMDoHDwVN2tF

6ulnRbATULdUE2CDIjNLvP

In [109]:
playlist = sp.user_playlist_tracks("spotify", "37i9dQZEVXbJwoKy8qKpHG")

In [110]:
playlist["total"] 

50

In [117]:
def get_name_artists_from_track(track):
    return [(track["name"],artist["name"]) for artist in track["artists"]]

In [115]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [111]:
def get_name_artist_from_playlist_item(playlist_item):
    return get_name_artists_from_track(playlist_item['track'])

In [133]:
def get_name_artists_from_playlist(input_playlist):
    return list(map(get_name_artist_from_playlist_item,input_playlist["items"]))

In [ ]:
def get_name_artists_from_playlist(input_playlist):
    return flatten(list(map(get_name_artist_from_playlist_item,input_playlist["items"])))

In [118]:
get_name_artists_from_playlist(playlist)

[('LA FAMA (with The Weeknd)', 'ROSALÍA'),
 ('LA FAMA (with The Weeknd)', 'The Weeknd'),
 ('Ateo', 'C. Tangana'),
 ('Ateo', 'Nathy Peluso'),
 ('Experimento', 'Myke Towers'),
 ('Mon Amour - Remix', 'zzoilo'),
 ('Mon Amour - Remix', 'Aitana'),
 ('Una Vaina Loca', 'Fuego'),
 ('Una Vaina Loca', 'Manuel Turizo'),
 ('Una Vaina Loca', 'Duki'),
 ('Anuel AA: Bzrp Music Sessions, Vol. 46', 'Bizarrap'),
 ('Anuel AA: Bzrp Music Sessions, Vol. 46', 'Anuel AA'),
 ('Nostálgico', 'Rvssian'),
 ('Nostálgico', 'Rauw Alejandro'),
 ('Nostálgico', 'Chris Brown'),
 ('Salimo de Noche', 'Tiago PZK'),
 ('Salimo de Noche', 'Trueno'),
 ('Ley Seca', 'Jhay Cortez'),
 ('Ley Seca', 'Anuel AA'),
 ('Dile (Homenaje)', 'Jhay Cortez'),
 ('Cúrame', 'Rauw Alejandro'),
 ('Tacones Rojos', 'Sebastian Yatra'),
 ('Todo De Ti', 'Rauw Alejandro'),
 ('Ptazeta: Bzrp Music Sessions, Vol. 45', 'Bizarrap'),
 ('Ptazeta: Bzrp Music Sessions, Vol. 45', 'Ptazeta'),
 ('El Incomprendido', 'Farruko'),
 ('El Incomprendido', 'Victor Cardenas'),

#### Using a big playlist to obtain a lot of songs by different artists

In [122]:
playlist = sp.user_playlist_tracks("spotify", "6ulnRbATULdUE2CDIjNLvP")

In [123]:
playlist["total"] 

805

In [124]:
# It is limited to 100 tracks, we will have to fix it:
len(playlist["items"])

100

In [125]:
playlist['next']

'https://api.spotify.com/v1/playlists/6ulnRbATULdUE2CDIjNLvP/tracks?offset=100&limit=100&additional_types=track'

In [ ]:
#sp.next(playlist)

In [136]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [137]:
all_tracks = get_playlist_tracks("6ulnRbATULdUE2CDIjNLvP")
len(all_tracks)

805

In [152]:
artist_and_track = [[],[]]
for i in range(len(all_tracks)):
    name_artists = ''
    for j in range(len(all_tracks[i]["track"]["artists"])):
        name_artists = name_artists + all_tracks[i]["track"]["artists"][j]["name"]
    artist_and_track[0].append(name_artists)
    artist_and_track[1].append(all_tracks[i]["track"]["name"])

In [156]:
different_artists = []
for i in range(len(all_tracks)):
    if artist_and_track[0][i] not in different_artists:
        different_artists = different_artists + [artist_and_track[0][i]]

In [158]:
#different_artists

In [194]:
def obtaining_all_songs_from_all_artists(lista):
    list_with_songs = [[],[]]
    i = 0
    for artist in lista:
        results = sp.search(q=artist, limit=50)
        tracks = results['tracks']['items']
        stop = 0
        while (results['tracks']['next']!=None) and (stop != 1):
            try:
                results = sp.next(results['tracks'])
                tracks = tracks + results['tracks']['items']
                sleep(randint(1,3))
            except:
                stop = 1
        list_with_songs[0].append(artist)
        list_with_songs[1].append(tracks)
        i += 1
    return list_with_songs

In [196]:
final_list = obtaining_all_songs_from_all_artists(different_artists)

ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search?query=The+Animals&type=track&offset=1000&limit=50 with Params: {} returned 404 due to Not found.
ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search?query=Gorillaz&type=track&offset=1000&limit=50 with Params: {} returned 404 due to Not found.
ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search?query=DEVO&type=track&offset=1000&limit=50 with Params: {} returned 404 due to Not found.
ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search?query=The+Black+Keys&type=track&offset=1000&limit=50 with Params: {} returned 404 due to Not found.
ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search?query=Cake&type=track&offset=1000&limit=50 with Params: {} returned 404 due to Not found.
ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/search?query=Franz+Ferdinand&type=track&offset=1000&limit=50 with Params: 

In [211]:
final_list[1][0][0]

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3ICflSq6ZgYAIrm2CTkfVP'},
    'href': 'https://api.spotify.com/v1/artists/3ICflSq6ZgYAIrm2CTkfVP',
    'id': '3ICflSq6ZgYAIrm2CTkfVP',
    'name': 'The Animals',
    'type': 'artist',
    'uri': 'spotify:artist:3ICflSq6ZgYAIrm2CTkfVP'}],
  'available_markets': ['CA', 'US'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/0Xwkx2TLFwRcoK0OWQ0Msa'},
  'href': 'https://api.spotify.com/v1/albums/0Xwkx2TLFwRcoK0OWQ0Msa',
  'id': '0Xwkx2TLFwRcoK0OWQ0Msa',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b2733c534611bd3658006378a2d7',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e023c534611bd3658006378a2d7',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d000048513c534611bd3658006378a2d7',
    'width': 64}],
  'name': 'The Animals',
  'release_date': '1964-01-01',
  'r

In [220]:
animals_songs = []
for i in range(len(final_list[1][0])):
    animals_songs = animals_songs + [final_list[1][0][i]['name']]
the_animals = pd.DataFrame(animals_songs)

In [221]:
the_animals

,0
0,House Of The Rising Sun
1,It's My Life
2,The Other Side Of Paradise
3,When I Was Young
4,House Of The Rising Sun
...,...
995,Animals An Animal
996,She Said Yeah
997,She's Attracted To
998,Animals Garden


In [213]:
final_list[1][0][0]['name']

'House Of The Rising Sun'